# Training and packaging Spacy

- In this notebook, we assume that we already have a usable vector space, contained in the `embeddings.txt` file. To see how we compile such a vector space, see the notebook `02-fastext-vectors.ipynb`

## Imports and utilities

In [1]:
import os
import spacy
import json

In [2]:
def make_dirs(path_or_uri: str, exist_ok: bool = True):
    return os.makedirs(path_or_uri, exist_ok=exist_ok)

## Configuration

In [3]:
lang = 'nb'
n_dims = 300               # vector dimensions
n_vectors = 10000          # in production we use 50k
vector_arch = 'fasttext'   # just used in model name
version = '2.0.0'
batch_size = 2000
patience = 1200            # 10000 better for real modelling
                           # very important parameter: how long SpaCy will run the training process before early stopping
                           # we do not know how long it will really take...

pkg_name_short = f'nhst_{vector_arch}_{n_dims}_{n_vectors}'
pkg_name_full = f'{lang}_{pkg_name_short}-{version}'

vectors_dir = f'nhst_{n_dims}'
job_dir = f'job_data'

# ready to use GPU: install pytorch in environment
use_gpu = False
if use_gpu:
    gpu_id = 0
    gpu_allocator = '"pytorch"'  # double quotes needed
    spacy.require_gpu()
else:
    gpu_id = -1
    gpu_allocator = 'none'

## Init vectors

In [4]:
# we already have a simple vector file... this should be computed in advance!
embeddings_local = 'embeddings.txt'

make_dirs(vectors_dir)

# run spacy init vectors
args = ['python', '-m', 'spacy', 'init', 'vectors', lang, embeddings_local, vectors_dir, '--prune', str(n_vectors),
        '--name', f'nhst_{n_dims}']

print(' '.join(args))  # copy output and run

python -m spacy init vectors nb embeddings.txt nhst_300 --prune 10000 --name nhst_300


In [5]:
!python -m spacy init vectors nb embeddings.txt nhst_300 --prune 5000 --name nhst_300

ℹ Creating blank nlp object for language 'nb'
20000it [00:00, 22055.90it/s]
✔ Successfully converted 5000 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting in [initialize].
/Users/emiliano/Development/spacy_norwegian_training_test/nhst_300


## Write job-setup files 

In [6]:
yml_config_str = '''
title: "Norwegian POS Tagging, Dependency Parsing (Universal Dependencies) and NER with Norne"
description: "Template to train a POS tagger, morphologizer, dependency parser amd named entity recogniser from a
[Universal Dependencies](https://universaldependencies.org/) corpus, in its Norne version.
It takes care of downloading the treebank, converting it to spaCy's format and training and evaluating the model."

vars:
  config: "default"
  lang: "{package_lang}"
  treebank: "norne"
  train_name: "no_bokmaal-ud-train"
  dev_name: "no_bokmaal-ud-dev"
  test_name: "no_bokmaal-ud-test"
  package_name: "{package_name}"
  package_version: "{package_version}"
  gpu: {gpu}

# These are the directories that the project needs. The project CLI will make sure that they always exist.
directories: ["assets", "corpus", "training", "metrics", "configs", "packages"]

assets:
  - dest: "assets/${{vars.treebank}}"
    git:
      repo: "https://github.com/ltgoslo/${{vars.treebank}}"  # "https://github.com/UniversalDependencies/${{vars.treebank}}"
      branch: "master"
      path: ""

workflows:
  all:
    - preprocess
    - train
    - evaluate
    - package

commands:
  - name: preprocess
    help: "Convert the data to spaCy's format"
    script:
      - "mkdir -p corpus/${{vars.treebank}}"
      - "python -m spacy convert assets/${{vars.treebank}}/ud/nob/${{vars.train_name}}.conllu corpus/${{vars.treebank}}/ --converter conllu --n-sents 10 --merge-subtokens"
      - "python -m spacy convert assets/${{vars.treebank}}/ud/nob/${{vars.dev_name}}.conllu corpus/${{vars.treebank}}/ --converter conllu --n-sents 10 --merge-subtokens"
      - "python -m spacy convert assets/${{vars.treebank}}/ud/nob/${{vars.test_name}}.conllu corpus/${{vars.treebank}}/ --converter conllu --n-sents 10 --merge-subtokens"
      - "mv corpus/${{vars.treebank}}/${{vars.train_name}}.spacy corpus/${{vars.treebank}}/train.spacy"
      - "mv corpus/${{vars.treebank}}/${{vars.dev_name}}.spacy corpus/${{vars.treebank}}/dev.spacy"
      - "mv corpus/${{vars.treebank}}/${{vars.test_name}}.spacy corpus/${{vars.treebank}}/test.spacy"
    deps:
      - "assets/${{vars.treebank}}/ud/nob/${{vars.train_name}}.conllu"
      - "assets/${{vars.treebank}}/ud/nob/${{vars.dev_name}}.conllu"
      - "assets/${{vars.treebank}}/ud/nob/${{vars.test_name}}.conllu"
    outputs:
      - "corpus/${{vars.treebank}}/train.spacy"
      - "corpus/${{vars.treebank}}/dev.spacy"
      - "corpus/${{vars.treebank}}/test.spacy"

  - name: train
    help: "Train ${{vars.treebank}}"
    script:
      - "python -m spacy train configs/${{vars.config}}.cfg --output training/${{vars.treebank}} --gpu-id ${{vars.gpu}} --paths.train corpus/${{vars.treebank}}/train.spacy --paths.dev corpus/${{vars.treebank}}/dev.spacy --nlp.lang=${{vars.lang}}"
    deps:
      - "corpus/${{vars.treebank}}/train.spacy"
      - "corpus/${{vars.treebank}}/dev.spacy"
      - "configs/${{vars.config}}.cfg"
    outputs:
      - "training/${{vars.treebank}}/model-best"

  - name: evaluate
    help: "Evaluate on the test data and save the metrics"
    script:
      - "python -m spacy evaluate ./training/${{vars.treebank}}/model-best ./corpus/${{vars.treebank}}/test.spacy --output ./metrics/metrics.json --gpu-id ${{vars.gpu}}"
    deps:
      - "training/${{vars.treebank}}/model-best"
      - "corpus/${{vars.treebank}}/test.spacy"
    outputs:
      - "metrics/metrics.json"

  - name: package
    help: "Package the trained model so it can be installed"
    script:
      - "python -m spacy package training/${{vars.treebank}}/model-best packages --name ${{vars.package_name}} --version ${{vars.package_version}} --force"
    deps:
      - "training/${{vars.treebank}}/model-best"
    outputs_no_cache:
      - "packages/${{vars.lang}}_${{vars.package_name}}-${{vars.package_version}}/dist/${{vars.lang}}_${{vars.package_name}}-${{vars.package_version}}.tar.gz"

  - name: clean
    help: "Remove intermediate files"
    script:
      - "rm -rf training/*"
      - "rm -rf metrics/*"
      - "rm -rf corpus/*"

'''

In [7]:
# Based on English and Norwegian official packages, plus GPU, plus static vectors and training tweaks
# 512 to 10k batch size, 0.15 dropout, 10k patience, larger matrix sizes, 6k tok2vec layer, eval 1k
exp_config_str = '''
[paths]
train = "corpus/norne/train.spacy"
dev = "corpus/norne/dev.spacy"
vectors = "../{vectors_dir}"
raw = null
init_tok2vec = null
vocab_data = null

[system]
gpu_allocator = {allocator}
seed = 0

[nlp]
lang = "nb"
pipeline = ["tok2vec","morphologizer","parser","senter","attribute_ruler","lemmatizer","ner"]
disabled = ["senter"]
before_creation = null
after_creation = null
after_pipeline_creation = null
batch_size = {batch_size}
tokenizer = {{"@tokenizers":"spacy.Tokenizer.v1"}}

[components]

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = ${{components.tok2vec.model.encode:width}}
attrs = ["NORM","PREFIX","SUFFIX","SHAPE"]
rows = [6000,3000,3000,3000]
include_static_vectors = true

[components.tok2vec.model.encode]
@architectures = "spacy.MaxoutWindowEncoder.v2"
width = 96
depth = 4
window_size = 1
maxout_pieces = 3

[components.parser]
factory = "parser"
learn_tokens = false
min_action_freq = 30
moves = null
update_with_oracle_cut_size = 100

[components.parser.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "parser"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.parser.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${{components.tok2vec.model.encode:width}}
upstream = "tok2vec"

[components.ner]
factory = "ner"
moves = null
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2Vec.v2"

[components.ner.model.tok2vec.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = 96
attrs = ["NORM","PREFIX","SUFFIX","SHAPE"]
rows = [6000,3000,3000,3000]
include_static_vectors = true

[components.ner.model.tok2vec.encode]
@architectures = "spacy.MaxoutWindowEncoder.v2"
width = 96
depth = 4
window_size = 1
maxout_pieces = 3

[components.morphologizer]
factory = "morphologizer"

[components.morphologizer.model]
@architectures = "spacy.Tagger.v1"
nO = null

[components.morphologizer.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${{components.tok2vec.model.encode:width}}
upstream = "tok2vec"

[components.attribute_ruler]
factory = "attribute_ruler"
validate = false

[components.lemmatizer]
factory = "lemmatizer"
mode = "rule"
model = null
overwrite = false

[components.senter]
factory = "senter"

[components.senter.model]
@architectures = "spacy.Tagger.v1"
nO = null

[components.senter.model.tok2vec]
@architectures = "spacy.Tok2Vec.v2"

[components.senter.model.tok2vec.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = 16
attrs = ["NORM","PREFIX","SUFFIX","SHAPE"]
rows = [1200,600,600,600]
include_static_vectors = true

[components.senter.model.tok2vec.encode]
@architectures = "spacy.MaxoutWindowEncoder.v2"
width = 16
depth = 2
window_size = 1
maxout_pieces = 2

[corpora]

[corpora.train]
@readers = "spacy.Corpus.v1"
path = ${{paths:train}}
max_length = 5000
gold_preproc = false
limit = 0

[corpora.train.augmenter]
@augmenters = "spacy.lower_case.v1"
level = 0.1

[corpora.dev]
@readers = "spacy.Corpus.v1"
limit = 0
max_length = 0
path = ${{paths:dev}}
gold_preproc = false
augmenter = null

[training]
train_corpus = "corpora.train"
dev_corpus = "corpora.dev"
seed = ${{system:seed}}
gpu_allocator = ${{system:gpu_allocator}}
dropout = 0.15
accumulate_gradient = 1
patience = {patience}
max_epochs = 0
max_steps = 0
eval_frequency = 1000
frozen_components = []
before_to_disk = null

[training.batcher]
@batchers = "spacy.batch_by_words.v1"
discard_oversize = false
tolerance = 0.2
get_length = null

[training.batcher.size]
@schedules = "compounding.v1"
start = 100
stop = 1000
compound = 1.001
t = 0.0

[training.logger]
@loggers = "spacy.ConsoleLogger.v1"
progress_bar = false

[training.optimizer]
@optimizers = "Adam.v1"
beta1 = 0.9
beta2 = 0.999
L2_is_weight_decay = true
L2 = 0.01
grad_clip = 1.0
use_averages = true
eps = 0.00000001
learn_rate = 0.001

[training.score_weights]
pos_acc = 0.16
morph_acc = 0.08
morph_per_feat = null
dep_uas = 0.0
dep_las = 0.18
dep_las_per_type = null
sents_p = null
sents_r = null
sents_f = 0.04
lemma_acc = 0.14
ents_f = 0.3
ents_p = 0.1
ents_r = 0.0
ents_per_type = null

[pretraining]

[initialize]
vocab_data = ${{paths.vocab_data}}
vectors = ${{paths.vectors}}
init_tok2vec = ${{paths.init_tok2vec}}
before_init = null
after_init = null

[initialize.components]

[initialize.components.morphologizer]

[initialize.components.morphologizer.labels]
@readers = "spacy.read_labels.v1"
path = "corpus/labels/morphologizer.json"
require = false

[initialize.components.ner]

[initialize.components.ner.labels]
@readers = "spacy.read_labels.v1"
path = "corpus/labels/ner.json"
require = false

[initialize.components.parser]

[initialize.components.parser.labels]
@readers = "spacy.read_labels.v1"
path = "corpus/labels/parser.json"
require = false

[initialize.lookups]
@misc = "spacy.LookupsDataLoader.v1"
lang = ${{nlp.lang}}
tables = []

[initialize.tokenizer]
'''

In [8]:
# save configuration strings to files
make_dirs(job_dir)
with open(f'{job_dir}/project.yml', 'w') as f:
    f.write(yml_config_str.format(package_lang=lang, package_name=pkg_name_short,
                                  package_version=version, gpu=gpu_id))

make_dirs(f'{job_dir}/configs')
with open(f'{job_dir}/configs/default.cfg', 'w') as f:
    f.write(exp_config_str.format(vectors_dir=vectors_dir, allocator=gpu_allocator,
                                  batch_size=batch_size, patience=patience))

## Downloading data assets for training

In [9]:
args = ['python', '-m', 'spacy', 'project', 'assets', job_dir]

print(' '.join(args))  # copy output and run

python -m spacy project assets job_data


In [10]:
!python -m spacy project assets job_data

ℹ Fetching 1 asset(s)
✔ Downloaded asset
/Users/emiliano/Development/spacy_norwegian_training_test/job_data/assets/norne


## Running job tasks

In [11]:
args = ['python', '-m', 'spacy', 'project', 'run', 'all', job_dir]

print(' '.join(args))  # copy output and run

python -m spacy project run all job_data


In [12]:
!python -m spacy project run all job_data

ℹ Running workflow 'all'

================================= preprocess =================================
ℹ Skipping 'preprocess': nothing changed

=================================== train ===================================
Running command: /Users/emiliano/Development/spacy_norwegian_training_test/env/bin/python -m spacy train configs/default.cfg --output training/norne --gpu-id -1 --paths.train corpus/norne/train.spacy --paths.dev corpus/norne/dev.spacy --nlp.lang=nb
ℹ Saving to output directory: training/norne
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'morphologizer', 'parser', 'attribute_ruler',
'lemmatizer', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS MORPH...  LOSS PARSER  LOSS NER  POS_ACC  MORPH_ACC  DEP_UAS  DEP_LAS  SENTS_F  LEMMA_ACC  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  --

## Saving output

- the necessary files are already saved, but we should move them to the final location

In [13]:
# built automatically by Spacy, but we need it to copy the package
pkg_file = f'{job_dir}/packages/{pkg_name_full}/dist/{pkg_name_full}.tar.gz'
print(pkg_file)

# evaluation metrics
metrics_file = f'{job_dir}/metrics/metrics.json'
print(metrics_file)

job_data/packages/nb_nhst_fasttext_300_10000-2.0.0/dist/nb_nhst_fasttext_300_10000-2.0.0.tar.gz
job_data/metrics/metrics.json


In [14]:
# check metrics
with open(f'{job_dir}/metrics/metrics.json') as f:
    metrics = json.load(f)
    for k, v in metrics.items():
        print('##', k, '##')
        print(v)
        print()

## token_acc ##
0.9975565671

## token_p ##
0.9974896238

## token_r ##
0.9944937596

## token_f ##
0.9959894389

## pos_acc ##
0.9650153089

## morph_acc ##
0.9469792033

## morph_micro_p ##
0.9690645618

## morph_micro_r ##
0.9606014503

## morph_micro_f ##
0.9648144473

## morph_per_feat ##
{'Definite': {'p': 0.9721749006, 'r': 0.9647788109, 'f': 0.9684627351}, 'Gender': {'p': 0.9307011414, 'r': 0.9194569094, 'f': 0.9250448573}, 'Number': {'p': 0.9689208895, 'r': 0.9580066685, 'f': 0.9634328696}, 'Mood': {'p': 0.9827288428, 'r': 0.9810344828, 'f': 0.9818809318}, 'Tense': {'p': 0.9838009835, 'r': 0.98266397, 'f': 0.983232148}, 'VerbForm': {'p': 0.9750869061, 'r': 0.9689119171, 'f': 0.9719896044}, 'Degree': {'p': 0.9592559787, 'r': 0.954185022, 'f': 0.9567137809}, 'Animacy': {'p': 0.9990176817, 'r': 0.9941348974, 'f': 0.9965703087}, 'Case': {'p': 0.988178025, 'r': 0.9827109267, 'f': 0.9854368932}, 'Person': {'p': 0.9912758997, 'r': 0.9885807504, 'f': 0.9899264906}, 'PronType': {'p': 0